In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow

data = pd.read_csv('Data_final.csv')

# Mostrar las primeras filas del dataset para familiarizarse con él
data.head()

,cole_bilingue,estu_genero,cole_calendario,cole_naturaleza,fami_educacionmadre,fami_educacionpadre,fami_estratovivienda,fami_tienecomputador,fami_tieneinternet,fami_tieneautomovil,desemp_ingles,punt_ingles,punt_matematicas,punt_sociales_ciudadanas,punt_c_naturales,punt_lectura_critica,punt_global
0,No,F,A,OFICIAL,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) incompleta,Estrato 2,Si,Si,Si,A1,37.0,50.0,39.0,41.0,46.0,217.0
1,No,M,A,OFICIAL,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) incompleta,Estrato 3,Si,Si,Si,A1,52.0,43.0,44.0,50.0,56.0,243.0
2,No,F,A,OFICIAL,Técnica o tecnológica completa,Técnica o tecnológica completa,Estrato 3,Si,Si,Si,A2,64.0,59.0,60.0,51.0,62.0,292.0
3,No,M,A,NO OFICIAL,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,Estrato 2,Si,Si,No,A2,60.0,64.0,62.0,55.0,61.0,302.0
4,No,F,A,NO OFICIAL,Educación profesional completa,Educación profesional incompleta,Estrato 3,Si,Si,Si,A1,52.0,52.0,49.0,48.0,53.0,253.0


In [36]:
# Definir las características (X) y la variable objetivo (y)
X = data.drop(columns=['punt_global','punt_ingles','punt_matematicas','punt_sociales_ciudadanas','punt_c_naturales','punt_lectura_critica','cole_bilingue','estu_genero','cole_calendario'])
y = data['punt_global']

# Identificar columnas categóricas y numéricas
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['number']).columns.tolist()

# Crear un preprocesador para transformar las columnas categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ]
)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicar el preprocesamiento a los datos
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Convertir a matrices densas
X_train = X_train.toarray()
X_test = X_test.toarray()

# Dividir el conjunto de entrenamiento en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Verificar la forma de los conjuntos de entrenamiento, validación y prueba
X_train.shape, X_val.shape, X_test.shape

((75886, 40), (18972, 40), (23715, 40))

In [37]:
# Inicializar MLflow
mlflow.set_experiment("Predicción de Puntaje Global Saber 11")

# Definir los hiperparámetros a probar
param_grid = {
    "learning_rate": [0.01, 0.001, 0.05],
    "epochs": [5, 15, 50 ,100],
    "batch_size": [32, 64],
    "layer_1_units": [128, 64],
    "layer_2_units": [64, 32],
    "layer_3_units": [32, 16]
}


In [38]:
def train_model(params, experiment_id):
    with mlflow.start_run(experiment_id=experiment_id):
        # Definir el modelo
        model = Sequential()
        model.add(Dense(params["layer_1_units"], input_dim=X_train.shape[1], activation='relu'))
        model.add(Dense(params["layer_2_units"], activation='relu'))
        model.add(Dense(params["layer_3_units"], activation='relu'))
        model.add(Dense(1, activation='linear'))
        
        # Compilar el modelo
        model.compile(optimizer=Adam(learning_rate=params["learning_rate"]), loss='mean_squared_error', metrics=['mean_absolute_error'])
        
        # Entrenar el modelo
        history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=params["epochs"], batch_size=params["batch_size"], verbose=1)
        
        # Evaluar el modelo en el conjunto de prueba
        y_pred = model.predict(X_test)
        
        # Calcular métricas de rendimiento
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)
        
        # Registrar parámetros en MLflow
        mlflow.log_params(params)
        
        # Registrar métricas en MLflow
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2", r2)
        
        # Graficar la historia de entrenamiento y registrar el gráfico
        plt.figure(figsize=(12, 6))
        plt.plot(history.history['loss'], label='Pérdida de Entrenamiento')
        plt.plot(history.history['val_loss'], label='Pérdida de Validación')
        plt.xlabel('Época')
        plt.ylabel('Pérdida')
        plt.legend()
        plt.title('Historia de Entrenamiento del Modelo')
        plt.savefig('loss_history.png')
        plt.close()
        mlflow.log_artifact('loss_history.png')
        
        # Graficar las predicciones vs los valores reales y registrar el gráfico
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--', color='red')
        plt.xlabel('Valores Reales')
        plt.ylabel('Predicciones')
        plt.title('Predicciones vs Valores Reales')
        plt.savefig('pred_vs_real.png')
        plt.close()
        mlflow.log_artifact('pred_vs_real.png')
        
        # Guardar el modelo en MLflow
        mlflow.keras.log_model(model, "model")
        
        print(f'MAE: {mae}')
        print(f'MSE: {mse}')
        print(f'RMSE: {rmse}')
        print(f'R²: {r2}')


In [41]:
input_dim=X_train.shape[1]
input_dim

40

In [39]:
# Probar diferentes combinaciones de hiperparámetros
for lr in param_grid["learning_rate"]:
    for epochs in param_grid["epochs"]:
        for batch_size in param_grid["batch_size"]:
            for l1_units in param_grid["layer_1_units"]:
                for l2_units in param_grid["layer_2_units"]:
                    for l3_units in param_grid["layer_3_units"]:
                        params = {
                            "learning_rate": lr,
                            "epochs": epochs,
                            "batch_size": batch_size,
                            "layer_1_units": l1_units,
                            "layer_2_units": l2_units,
                            "layer_3_units": l3_units
                        }
                        train_model(params, experiment.experiment_id)



Epoch 1/5
2372/2372 [==============================] - 6s 2ms/step - loss: 1503.3107 - mean_absolute_error: 28.4985 - val_loss: 1066.0344 - val_mean_absolute_error: 26.0619
Epoch 2/5
2372/2372 [==============================] - 4s 2ms/step - loss: 1113.9531 - mean_absolute_error: 26.6887 - val_loss: 1058.2676 - val_mean_absolute_error: 25.9674
Epoch 3/5
2372/2372 [==============================] - 4s 2ms/step - loss: 1093.2545 - mean_absolute_error: 26.4566 - val_loss: 1050.2509 - val_mean_absolute_error: 26.0177
Epoch 4/5
2372/2372 [==============================] - 4s 2ms/step - loss: 1070.1703 - mean_absolute_error: 26.1774 - val_loss: 1039.7289 - val_mean_absolute_error: 25.7950
Epoch 5/5
742/742 [==============================] - 1s 1ms/step


2024/05/30 04:34:56 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\57313\AppData\Local\Temp\tmpaultbecb\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\57313\AppData\Local\Temp\tmpaultbecb\model\data\model\assets


MAE: 26.145546517642572
MSE: 1086.4241649511723
RMSE: 32.96094909057038
R²: 0.5558151485690561
Epoch 1/5
2372/2372 [==============================] - 6s 2ms/step - loss: 1498.1534 - mean_absolute_error: 28.2978 - val_loss: 1136.7086 - val_mean_absolute_error: 26.8263
Epoch 2/5
2372/2372 [==============================] - 5s 2ms/step - loss: 1096.8848 - mean_absolute_error: 26.4930 - val_loss: 1036.0128 - val_mean_absolute_error: 25.8989
Epoch 3/5
2372/2372 [==============================] - 5s 2ms/step - loss: 1083.3494 - mean_absolute_error: 26.3415 - val_loss: 1092.5166 - val_mean_absolute_error: 26.2930
Epoch 4/5
2372/2372 [==============================] - 5s 2ms/step - loss: 1068.0618 - mean_absolute_error: 26.1511 - val_loss: 1039.4691 - val_mean_absolute_error: 25.7703
Epoch 5/5
742/742 [==============================] - 1s 1ms/step


...

2024/05/30 11:13:08 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\57313\AppData\Local\Temp\tmp8mjk60_4\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\57313\AppData\Local\Temp\tmp8mjk60_4\model\data\model\assets


MAE: 25.695091276834365
MSE: 1029.9433528728014
RMSE: 32.092730530025044
R²: 0.5789073458259699
Epoch 1/100
1186/1186 [==============================] - 4s 2ms/step - loss: 1950.4579 - mean_absolute_error: 29.7431 - val_loss: 1042.5745 - val_mean_absolute_error: 25.8437
Epoch 2/100
1186/1186 [==============================] - 3s 2ms/step - loss: 1077.0350 - mean_absolute_error: 26.2419 - val_loss: 1092.0607 - val_mean_absolute_error: 26.3114
Epoch 3/100
1186/1186 [==============================] - 3s 2ms/step - loss: 1070.8639 - mean_absolute_error: 26.1727 - val_loss: 1046.4025 - val_mean_absolute_error: 25.8784
Epoch 4/100
1186/1186 [==============================] - 3s 2ms/step - loss: 1063.3481 - mean_absolute_error: 26.0916 - val_loss: 1048.5144 - val_mean_absolute_error: 25.8690
Epoch 5/100
1186/1186 [==============================] - 2s 2ms/step - loss: 1059.4596 - mean_absolute_error: 26.0481 - val_loss: 1036.5983 - val_mean_absolute_error: 25.8933
Epoch 6/100
1186/1186 [======

KeyboardInterrupt: 